# Molecular Dynamics
This Jupyter Notebook provides a step-by-step guide for performing a Molecular Dynamics (MD) simulation using GROMACS. Initially, dependencies such as GROMACS and Python libraries are installed. Subsequently, the initial molecular structure is preprocessed, solvated, and various simulation parameters are set up. Multiple stages of the MD simulation, including energy minimization, equilibration, and production MD, are then executed. Lastly, trajectory analysis is performed, including the calculation of parameters like RMSD (Root Mean Square Deviation) and gyration, offering insights into the molecular system's behavior over time. This notebook is employed as a comprehensive tool for simulating and analyzing the molecular dynamics of the specified system.

## Input/Output Description
- **Input:**
1. A PDB file (`maws_pfoa_aptamer.pdb`) representing the initial molecular structure.
1. GROMACS parameter and configuration files (`ions.mdp`, `minim.mdp`, `nvt.mdp`, `npt.mdp`, `md.mdp`) for different simulation stages.
`ions.mdp`: parameters for simulating the addition of ions to the system to achieve a desired salt concentration.
`minim.mdp`: energy minimization, which aims to minimize the energy of the system to achieve a stable starting point for simulations.
`nvt.mdp`: parameters for an NVT ensemble simulation, where the number of particles, volume, and temperature of the system are held constant.
`npt.mdp`: NPT ensemble simulations, where the system is equilibrated at constant temperature and pressure.
`md.mdp`: main production phase of the MD simulation, specifying simulation parameters for data collection on the system's dynamics.
- **Output:**
1. Processed and solvated molecular structures in GROMACS formats.
1. Energy, temperature, pressure, and density profiles during the simulation.
1. Trajectory and analysis files including RMSD, gyration, and more.

### REMEMBER: change the Jupyter kernel to **Python (AptaLoop)**. See our README file for detailed information.


In [ ]:
%%bash
pip install --upgrade pip
pip install wheel
pip install setuptools
pip install --upgrade setuptools wheel
pip install --upgrade cmake pybind11
pip install nglview
pip install GromacsWrapper

In [ ]:
import subprocess
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np 
import gromacs 
import gromacs.formats
import nglview as nv
import os

### Replace `maws_pfoa_aptamer.pdb` with the path of your file (relative to the script path) in the following cell

In [1]:
script_path = os.getcwd()
input_file = os.path.join(script_path, "maws_pfoa_aptamer.pdb")

In [ ]:
!grep -v HOH $input_file > python_aptamer.pdb
!gmx pdb2gmx -f python_aptamer.pdb -o python_aptamer_processed.gro -ignh -ff amber03 -water tip3p
!gmx editconf -f python_aptamer_processed.gro -o python_newbox.gro -c -d 0.5 -bt cubic
!gmx solvate -cp python_newbox.gro -cs spc216.gro -o python_solv.gro -p topol.top
!gmx grompp -f ions.mdp -c python_solv.gro -p topol.top -o ions.tpr
!printf "SOL" | gmx genion -s ions.tpr -o python_solv_ions.gro -p topol.top -pname NA -nname CL -neutral
!gmx grompp -f minim.mdp -c python_solv_ions.gro -p topol.top -o em.tpr
!gmx mdrun -v -deffnm em
!printf "10 0"| gmx energy -f em.edr -o potential.xvg
!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
!gmx mdrun -deffnm nvt
!printf "16 0" | gmx energy -f nvt.edr -o temperature.xvg
!gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
!gmx mdrun -deffnm npt
!printf "18 0" | gmx energy -f npt.edr -o pressure.xvg
!gmx energy -f npt.edr -o density.xvg
!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_python.tpr
!gmx mdrun -deffnm md_python

In [ ]:
!printf "1 0" | gmx trjconv -s md_python.tpr -f md_python.xtc -o md_python_noPBC.xtc -pbc mol -center
!printf "4 0" | gmx rms -s md_0_1.tpr -f md_python_noPBC.xtc -o rmsd.xvg -tu ns
!gmx rms -s em.tpr -f md_python_noPBC.xtc -o rmsd_xtal.xvg -tu ns
!printf "1 0" | gmx gyrate -s md_python.tpr -f md_python_noPBC.xtc -o gyrate.xvg